# Feature

- stage average
- average difference 
- stage peak
- stage activation 
- begin slope (trail, rest)
- stadard deviation
- std difference   

base (5s) -> trail (30s) -> feedback (5s) -> rest (20s)

## Module

In [3]:
import os
import glob
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
import scipy.signal as signal
import seaborn as sns

In [4]:
# 5秒rest, 30秒trail time, 5秒回饋, 20秒recovery
t_base = 5
t_trail = 35
t_feedback = 40
t_rest = 60
trail_times = 23
transfer_times = 8

## Stage Average
`df[t0:t1].mean()`

In [36]:
def stage_ave(df, plot=False, save=False):
    '''
    return: the average of each stage and difference of average of between two stages

    inputs:
        df: dataframe
        plot: plot the figure or not
        save: save the figure or not, figure not shown in code if true
    '''
    # Calculations
    Base_ave = df[0:t_base].mean()
    Trail_ave = df[t_base:t_trail].mean()
    Feedback_ave = df[t_trail:t_feedback].mean()
    Rest_ave = df[t_feedback:t_rest].mean()
    Base_Trail_diff = Trail_ave - Base_ave
    Trail_Feedback_diff = Feedback_ave - Trail_ave
    Feedback_Rest_diff = Rest_ave - Feedback_ave

    # rename col_name
    Base_ave = Base_ave.rename(index = lambda col: col+'_Base_Ave')
    Trail_ave = Trail_ave.rename(index = lambda col: col+'_Trail_Ave')
    Feedback_ave = Feedback_ave.rename(index = lambda col: col+'_Feedback_Ave')
    Rest_ave = Rest_ave.rename(index = lambda col: col+'_Rest_Ave')
    Base_Trail_diff = Base_Trail_diff.rename(index = lambda col: col+'_Base_Trail_Diff')
    Trail_Feedback_diff = Trail_Feedback_diff.rename(index = lambda col: col+'_Trail_Feedback_Diff')
    Feedback_Rest_diff = Feedback_Rest_diff.rename(index = lambda col: col+'_Feedback_Rest_Diff')

    # concat all series
    Stage_Ave = pd.concat([Base_ave, Trail_ave, Feedback_ave, Rest_ave, Base_Trail_diff, Trail_Feedback_diff, Feedback_Rest_diff], axis=0)

    # plot
    if plot == True:
        sns.set_theme(style='whitegrid')
        sns.set(font_scale=1)
        for ch in range(2,3):
            plt.figure(figsize=(12, 5))
            sns.lineplot(x=df.index, y=df['CH'+str(ch)+'_Oxy'], data=df, color='tab:red', label= 'Oxy', linewidth = 2.5)
            plt.axvspan(0, t_base, facecolor='white', alpha=0.2) 
            plt.axvspan(t_base, t_trail, facecolor='lightcoral', alpha=0.2)
            plt.axvspan(t_trail, t_feedback, facecolor='white', alpha=0.2)
            plt.axvspan(t_feedback, t_rest, facecolor='steelblue', alpha=0.2)
            plt.plot([t_base, t_trail], [Trail_ave['CH'+str(ch)+'_Oxy'], Trail_ave['CH'+str(ch)+'_Oxy']], color='k', linewidth = 2.5, linestyle='dashed')
            plt.plot([0, t_base], [Base_ave['CH'+str(ch)+'_Oxy'], Base_ave['CH'+str(ch)+'_Oxy']], color='k', linewidth = 2.5, linestyle='dashed')
            plt.annotate('', xy=(t_base, Base_ave['CH'+str(ch)+'_Oxy']), xytext=(t_base, Trail_ave['CH'+str(ch)+'_Oxy']), arrowprops=dict(color='black', arrowstyle='<->', linewidth=2.5)) # 
            plt.xlim([0, 60])
            plt.legend()
            plt.show()

    return Stage_Ave

## Stage Peak
`df[t0:t1].max()`

In [40]:
def stage_peak(df, plot=False, save=False):
    '''
    return: the peak of each stage and difference of peak of between two stages

    inputs:
        df: dataframe
        plot: plot the figure or not
        save: save the figure or not, figure not shown in code if true
    '''
    # Calculation
    Base_peak = df[0:t_base].max()
    Trail_peak = df[t_base:t_trail].max()
    Feedback_peak = df[t_trail:t_feedback].max()
    Rest_peak = df[t_feedback:t_rest].max()
    
    # rename col_name
    Base_peak = Base_peak.rename(index = lambda col: col+'_Base_Peak')
    Trail_peak = Trail_peak.rename(index = lambda col: col+'_Trail_Peak')
    Feedback_peak = Feedback_peak.rename(index = lambda col: col+'_Feedback_Peak')
    Rest_peak = Rest_peak.rename(index = lambda col: col+'_Rest_Peak')

    # concat all series
    Stage_Peak = pd.concat([Base_peak, Trail_peak, Feedback_peak, Rest_peak], axis=0)

    if plot == True:
        for ch in range(2,3):
            plt.figure(figsize=(12, 5))
            sns.lineplot(x=df.index, y=df['CH'+str(ch)+'_Oxy'], data=df, color='tab:red', label= 'Oxy', linewidth = 2.5)
            plt.axvspan(0, t_base, facecolor='white', alpha=0.2) 
            plt.axvspan(t_base, t_trail, facecolor='lightcoral', alpha=0.2)
            plt.axvspan(t_trail, t_feedback, facecolor='white', alpha=0.2)
            plt.axvspan(t_feedback, t_rest, facecolor='steelblue', alpha=0.2)
            plt.plot([t_base, t_trail], [Trail_peak['CH'+str(ch)+'_Oxy'], Trail_peak['CH'+str(ch)+'_Oxy']], color='k', linewidth = 2.5, linestyle='dashed')
            plt.plot([t_base, t_trail], [Trail_peak['CH'+str(ch)+'_Oxy'], Trail_peak['CH'+str(ch)+'_Oxy']], color='k', linewidth = 2.5, linestyle='dashed')
            plt.annotate('', xy=((t_trail-t_base)/2, 0), xytext=(t_base, Trail_peak['CH'+str(ch)+'_Oxy']), arrowprops=dict(color='black', arrowstyle='<->', linewidth=2.5)) # 
            plt.xlim([0, 60])
            plt.legend()
            plt.show()

    return Stage_Peak

## Stage Activation
`df[t0:t1].max() - df[t0:t1].min()`

In [7]:
def stage_activation(df, plot=False, save=False):
    '''
    return: the activarion of each stage and difference of peak of between two stages

    inputs:
        df: dataframe
        plot: plot the figure or not
        save: save the figure or not, figure not shown in code if true
    '''
    # Calculation
    Base_activation = df[0:t_base].max()-df[0:t_base].min()
    Trail_activation = df[t_base:t_trail].max()-df[t_base:t_trail].min()
    Feedback_activation = df[t_trail:t_feedback].max()-df[t_trail:t_feedback].min()
    Rest_activation = df[t_feedback:t_rest].max()-df[t_feedback:t_rest].min()
    
    # rename col_name
    Base_activation = Base_activation.rename(index = lambda col: col+'_Base_Activation')
    Trail_activation = Trail_activation.rename(index = lambda col: col+'_Trail_Activation')
    Feedback_activation = Feedback_activation.rename(index = lambda col: col+'_Feedback_Activation')
    Rest_activation = Rest_activation.rename(index = lambda col: col+'_Rest_Activation')

    # concat all series
    Stage_Activation = pd.concat([Base_activation, Trail_activation, Feedback_activation, Rest_activation], axis=0)

    if plot == True:
        for ch in range(2,3):
            plt.figure(figsize=(12, 5))
            sns.lineplot(x=df.index, y=df['CH'+str(ch)+'_Oxy'], data=df, color='tab:red', label= 'Oxy', linewidth = 2.5)
            plt.axvspan(0, t_base, facecolor='white', alpha=0.2) 
            plt.axvspan(t_base, t_trail, facecolor='lightcoral', alpha=0.2)
            plt.axvspan(t_trail, t_feedback, facecolor='white', alpha=0.2)
            plt.axvspan(t_feedback, t_rest, facecolor='steelblue', alpha=0.2)
            plt.plot([t_base, t_trail], [Trail_peak['CH'+str(ch)+'_Oxy'], Trail_peak['CH'+str(ch)+'_Oxy']], color='k', linewidth = 2.5, linestyle='dashed')
            plt.plot([t_base, t_trail], [Trail_peak['CH'+str(ch)+'_Oxy'], Trail_peak['CH'+str(ch)+'_Oxy']], color='k', linewidth = 2.5, linestyle='dashed')
            plt.annotate('', xy=((t_trail-t_base)/2, 0), xytext=(t_base, Trail_peak['CH'+str(ch)+'_Oxy']), arrowprops=dict(color='black', arrowstyle='<->', linewidth=2.5)) # 
            plt.xlim([0, 60])
            plt.legend()
            plt.show()

    return Stage_Activation

## Begin slope
`(df[t0+8] - df[t0]) / 8`

In [8]:
def begin_slope(df, plot=False, save=False):
    Trail_slope = (df[t_base:t_base+8].tail(1).reset_index(drop=True) -  df[t_base:t_base+8].head(1).reset_index(drop=True)) / 8
    Rest_slope = (df[t_feedback:t_feedback+8].tail(1).reset_index(drop=True) -  df[t_feedback:t_feedback+8].head(1).reset_index(drop=True)) / 8

    Trail_slope = Trail_slope.squeeze().rename(index = lambda col: col+'_Trail_Slope')
    Rest_slope = Rest_slope.squeeze().rename(index = lambda col: col+'_Rest_Slope')
    Stage_slope = pd.concat([Trail_slope, Rest_slope], axis=0)

    return Stage_slope

## Stadard deviation
`df[t0:t1].std()`

In [9]:
def std(df, plot=False, save=False):
    '''
    return: the Stadard deviation of each stage and difference of peak of between two stages

    inputs:
        df: dataframe
        plot: plot the figure or not
        save: save the figure or not, figure not shown in code if true
    '''
    # Calculation
    Base_std = df[0:t_base].std()
    Trail_std = df[t_base:t_trail].std()
    Feedback_std = df[t_trail:t_feedback].std()
    Rest_std = df[t_feedback:t_rest].std()
    Base_Trail_std_diff = Trail_std - Base_std
    Trail_Feedback_std_diff = Feedback_std - Trail_std
    Feedback_Rest_std_diff = Rest_std - Feedback_std
    
    # rename col_name
    Base_std = Base_std.rename(index = lambda col: col+'_Base_Std')
    Trail_std = Trail_std.rename(index = lambda col: col+'_Trail_Std')
    Feedback_std = Feedback_std.rename(index = lambda col: col+'_Feedback_Std')
    Rest_std = Rest_std.rename(index = lambda col: col+'_Rest_Std')
    Base_Trail_std_diff = Base_Trail_std_diff.rename(index = lambda col: col+'_Base_Trail_Std_Diff')
    Trail_Feedback_std_diff = Trail_Feedback_std_diff.rename(index = lambda col: col+'_Trail_Feedback_Std_Diff')
    Feedback_Rest_std_diff = Feedback_Rest_std_diff.rename(index = lambda col: col+'_Feedback_Rest_Std_Diff')
    
    # concat all series
    Stage_std = pd.concat([Base_std, Trail_std, Feedback_std, Rest_std, Base_Trail_std_diff, Trail_Feedback_std_diff, Feedback_Rest_std_diff], axis=0)

    return Stage_std

## Main

In [ ]:
path = 'C:/Users/User/Desktop/Github/TS/4_normalized_data/'
groups = ['High', 'Low']
for group in groups:
    os.chdir(path + group)
    files = glob.glob('*csv')
    Feature = pd.DataFrame()
    for file in files:
        print(file)
        OwO = pd.read_csv(file, index_col='Time')
        OwO = OwO.drop(columns=['CH1_Oxy', 'CH1_Deoxy', 'CH4_Oxy', 'CH4_Deoxy'])
        OwO['CH2_HbT'] = OwO['CH2_Oxy'] + OwO['CH2_Deoxy']
        OwO['CH3_HbT'] = OwO['CH3_Oxy'] + OwO['CH3_Deoxy']
        stage_ave = stage_ave(OwO)
        stage_peak = stage_peak(OwO)
        stage_activation = stage_activation(OwO)
        begin_slope = begin_slope(OwO)
        std = std(OwO)
        feature_temp =  pd.concat([stage_ave, stage_peak, stage_activation, begin_slope, std]).to_frame().T
        feature = pd.concat([Feature, feature_temp])
    feature.to_csv('C:/Users/User/Desktop/Github/TS/5_Feature/' + group + '_feature.csv', index=False)